### Find a perfect location to live in Toronto

In [3]:
import numpy as np # library to handle data in a vectorized manner
!pip install folium
!pip install bs4
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### Import Data

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,"html5lib")
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_2b9c9baf52924b05af3cd985b8c85eb4 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_2b9c9baf52924b05af3cd985b8c85eb4 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_2b9c9baf52924b05af3cd985b8c85eb4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fuDz640gsSlJxVkO6hxnk_3-ASVDFTnADhWMgDfeVeVh',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_2b9c9baf52924b05af3cd985b8c85eb4)

body = client_2b9c9baf52924b05af3cd985b8c85eb4.get_object(Bucket='coursera-donotdelete-pr-s9od4u4qe9yjsy',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()
data_with_cood = df.join(df_data_1.set_index('Postal Code'), on = 'PostalCode')

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
#set api
CLIENT_ID = 'WLLSNTD2EBPBQJQKUYY1XIWI4A01HJPJMZWYTBVNG5HPGD0E'
CLIENT_SECRET = 'OT0SIEZTCSNWPJEKBTC1TA2U3OKHMHXXQCVDNFJXOQ3S4LPX'
VERSION = 20180602
LIMIT = 100
near_by = getNearbyVenues(data_with_cood['Neighborhood'], data_with_cood['Latitude'], data_with_cood['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [10]:
near_by['Venue Category'].unique()

array(['Park', 'Food & Drink Shop', 'Hockey Arena',
       'Portuguese Restaurant', 'Coffee Shop', 'Intersection',
       'Pizza Place', 'Bakery', 'Distribution Center', 'Spa',
       'Restaurant', 'Breakfast Spot', 'Gym / Fitness Center',
       'Historic Site', 'Chocolate Shop', 'Farmers Market', 'Pub',
       'Performing Arts Venue', 'Dessert Shop', 'Yoga Studio',
       'Mexican Restaurant', 'Café', 'Theater', 'Event Space',
       'French Restaurant', 'Shoe Store', 'Art Gallery', 'Cosmetics Shop',
       'Asian Restaurant', 'Electronics Store', 'Bank', 'Beer Store',
       'Hotel', 'Health Food Store', 'Wine Shop', 'Antique Shop',
       'Playground', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       "Women's Store", 'Carpet Store', 'Miscellaneous Shop',
       'Italian Restaurant', 'Creperie', 'Beer Bar', 'Sushi Restaurant',
       'Burrito Place', 'Diner', 'Discount Store', 'Fried Chicken Joint',
       'Japanese 

### We first find the top 10 neighboorhood which have the most Restaurant near by. 

In [45]:
most_food = near_by[near_by["Venue Category"].str.contains("Restaurant")]
most_food = most_food.groupby('Neighborhood').count().sort_values(by = 'Venue', ascending = False)
most_food['food_rank'] = most_food['Venue'].rank(ascending = False)
most_food.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,food_rank
Neighborhood,,,,,,,
"First Canadian Place, Underground city",3,3,3,3,3,3,1.0
"Little Portugal, Trinity",2,2,2,2,2,2,2.5
"Toronto Dominion Centre, Design Exchange",2,2,2,2,2,2,2.5
"Commerce Court, Victoria Hotel",1,1,1,1,1,1,7.0
Don Mills South,1,1,1,1,1,1,7.0
"Fairview, Henry Farm, Oriole",1,1,1,1,1,1,7.0
Glencairn,1,1,1,1,1,1,7.0
"Regent Park, Harbourfront",1,1,1,1,1,1,7.0
"Richmond, Adelaide, King",1,1,1,1,1,1,7.0


### Now we find the top 10 neighboorhood which have the most Shops near by

In [51]:
most_shop = near_by[near_by["Venue Category"].str.contains("Shop")]
most_shop = most_shop.groupby('Neighborhood').count().sort_values(by = 'Venue', ascending = False)
most_shop['shop_rank'] = most_shop['Venue'].rank(ascending = False)
most_shop.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,shop_rank
Neighborhood,,,,,,,
Enclave of M5E,23,23,23,23,23,23,1.5
"Garden District, Ryerson",23,23,23,23,23,23,1.5
"Harbourfront East, Union Station, Toronto Islands",18,18,18,18,18,18,3.0
Central Bay Street,17,17,17,17,17,17,4.5
"Commerce Court, Victoria Hotel",17,17,17,17,17,17,4.5
"Toronto Dominion Centre, Design Exchange",15,15,15,15,15,15,6.5
"Richmond, Adelaide, King",15,15,15,15,15,15,6.5
Berczy Park,13,13,13,13,13,13,9.0
"Regent Park, Harbourfront",13,13,13,13,13,13,9.0


In [56]:
most_bar = near_by[near_by["Venue Category"].str.contains("Bar")]
most_bar = most_bar.groupby('Neighborhood').count().sort_values(by = 'Venue', ascending = False)
most_bar['bar_rank'] = most_bar['Venue'].rank(ascending = False)
most_bar.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,bar_rank
Neighborhood,,,,,,,
"Little Portugal, Trinity",8,8,8,8,8,8,2.0
"Commerce Court, Victoria Hotel",8,8,8,8,8,8,2.0
Enclave of M5E,8,8,8,8,8,8,2.0
"First Canadian Place, Underground city",7,7,7,7,7,7,5.5
Berczy Park,7,7,7,7,7,7,5.5
Church and Wellesley,7,7,7,7,7,7,5.5
"Harbourfront East, Union Station, Toronto Islands",7,7,7,7,7,7,5.5
"Kensington Market, Chinatown, Grange Park",6,6,6,6,6,6,8.5
St. James Town,6,6,6,6,6,6,8.5


In [57]:
most_cin = near_by[near_by["Venue Category"].str.contains("Movie Theater")]
most_cin = most_cin.groupby('Neighborhood').count().sort_values(by = 'Venue', ascending = False)
most_cin['cin_rank'] = most_cin['Venue'].rank(ascending = False)
most_cin.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,cin_rank
Neighborhood,,,,,,,
"Garden District, Ryerson",2,2,2,2,2,2,1.0
Enclave of M5E,1,1,1,1,1,1,5.0
"Fairview, Henry Farm, Oriole",1,1,1,1,1,1,5.0
"Harbourfront East, Union Station, Toronto Islands",1,1,1,1,1,1,5.0
"India Bazaar, The Beaches West",1,1,1,1,1,1,5.0
"Parkdale, Roncesvalles",1,1,1,1,1,1,5.0
"Runnymede, Swansea",1,1,1,1,1,1,5.0
Willowdale South,1,1,1,1,1,1,5.0


In [59]:
ans = pd.concat([most_food, most_shop, most_cin, most_bar], axis = 1)
ans[['shop_rank', 'cin_rank', 'bar_rank', 'food_rank']]

,shop_rank,cin_rank,bar_rank,food_rank
"First Canadian Place, Underground city",14.5,NaN,5.5,1.0
"Little Portugal, Trinity",17.0,NaN,2.0,2.5
"Toronto Dominion Centre, Design Exchange",6.5,NaN,10.0,2.5
"Commerce Court, Victoria Hotel",4.5,NaN,2.0,7.0
Don Mills South,21.5,NaN,NaN,7.0
"Fairview, Henry Farm, Oriole",9.0,5.0,11.0,7.0
Glencairn,NaN,NaN,NaN,7.0
"Regent Park, Harbourfront",9.0,NaN,NaN,7.0
"Richmond, Adelaide, King",6.5,NaN,12.5,7.0
St. James Town,12.5,NaN,8.5,7.0


We can see that neighborhood Fairview, Henry Farm, Oriole has all four category satisfied. However, given the fact that this client do not have a car, this area might not be the best fit for him. Now we reduce our standard of having 'Movie theater' and see the new results

In [68]:
new_ans = pd.concat([most_food, most_shop, most_bar], axis = 1)
new_ans['all_rank'] = new_ans['shop_rank'] + new_ans['bar_rank'] + new_ans['food_rank']
new_ans = new_ans.sort_values(by = 'all_rank')
new_ans[['shop_rank', 'bar_rank', 'food_rank', 'all_rank']].head()

,shop_rank,bar_rank,food_rank,all_rank
"Commerce Court, Victoria Hotel",4.5,2.0,7.0,13.5
"Toronto Dominion Centre, Design Exchange",6.5,10.0,2.5,19.0
"First Canadian Place, Underground city",14.5,5.5,1.0,21.0
"Little Portugal, Trinity",17.0,2.0,2.5,21.5
"Richmond, Adelaide, King",6.5,12.5,7.0,26.0


Here we have our candidate. Since the among all these area, Little Portugal is more suitable for living comparing to other area which mostly being in the financial district. 
We conclude that our clinet will be happy to live at Little Portugal, Trinity